In [ ]:
!pip install -q "protobuf==3.20.*"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.2 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptograp

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Bidirectional, Input
import json
import os
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

2025-12-09 09:10:35.228801: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765271435.426581      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765271435.476922      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
diacritic2id_path = "/kaggle/input/arabicia-2/diacritic2id.json"
idx2char_path = "/kaggle/input/arabicia-3/idx2char.json"
char2idx_path = "/kaggle/input/arabicia-3/char2idx.json"

y_train_path = "/kaggle/input/arabicia-3/cleaned_tashkeel.txt"
y_val_path = "/kaggle/input/arabicia-3/cleaned_tashkeel_val.txt"

padded_train = "/kaggle/input/arabicia-3/padded_2.pkl"
padded_val = "/kaggle/input/arabicia-3/padded_val.pkl"

char_embeddings_path = "/kaggle/input/arabic-chars/keras/default/1/embedding_matrix(1).npy"

In [ ]:
with open(diacritic2id_path, 'r', encoding='utf-8') as f:
    diacritic2id = json.load(f)

id2label = {v: k for k, v in diacritic2id.items()}

with open(char2idx_path, 'r', encoding='utf-8') as f:
    char2idx = json.load(f)

with open(idx2char_path, 'r', encoding='utf-8') as f:
    idx2char = json.load(f)

# idx2char['1'] = '?'
# value = char2idx.pop('\uFFFD')
# char2idx['?'] = value
for key, value in char2idx.items():
    if value != 0:
        char2idx[key] = value - 1

In [ ]:
def recover(sentence, tashkeel):
    result = ""
    i = 0
    while i < len(sentence):
        result += sentence[i] + tashkeel[i]
        i += 1

    return result

In [ ]:
def load_data_pickle(file_path):
    with open(file_path, 'rb') as f:
        X_raw, y_raw = pickle.load(f)
    X = []
    y = []

    for text_seq, label_seq in zip(X_raw, y_raw):
        x_ids = [c for c in text_seq]
        y_ids = [t for t in label_seq]

        X.append(x_ids)
        y.append(y_ids)

    return X, y

In [ ]:
sentences, tashkeel_sequences = load_data_pickle(padded_train)

In [ ]:
new_sentences = sentences
new_tashkeel = tashkeel_sequences
# for i in range(len(sentences)):
#     text, tashkeel = remove_pads(sentences[i], tashkeel_sequences[i])
#     new_sentences.append(text)
#     new_tashkeel.append(tashkeel)

In [ ]:
sentence_lengths = [len(sentence) for sentence in new_sentences]

In [ ]:
files_config = []
for i in range(0, 64001, 16000):
    files_config.append((f'/kaggle/input/arabic-diacrirization-features-electra-{i + 16000}/zizo_part_{i + 16000}.h5', sentence_lengths[i:i + 16000]))
files_config.append((f'/kaggle/input/arabic-diacrirization-features-electra-80392/zizo_part_80392.h5', sentence_lengths[80000:80392]))

In [ ]:
MAX_SEQ_LEN = 807
CHAR_EMBED_DIM = 256
BATCH_SIZE = 32
EPOCHS = 10
VOCAB_SIZE = len(char2idx)
NUM_CLASSES = len(diacritic2id)
DIRTY = False
DIACRITICS_PATTERN = re.compile(r'[\u064B-\u0652]')
INPUT_DIM = 1024 # bert + mine
HIDDEN_DIM = 512
total_sentences = sum([len(lengths) for _, lengths in files_config])
STEPS_PER_EPOCH = total_sentences // BATCH_SIZE
PADDING_INPUT = -99999.0

In [ ]:
custom_char_embedding = np.load('/kaggle/input/arabic-chars/keras/default/1/embedding_matrix(1).npy')

def extract_custom_char_embeddings(char):
    return custom_char_embedding[char2idx[char]]

In [ ]:
punctuation = ['.', ':', '{', '}', '[', ']', '(', ')', '؛', '«', '»', '!', '،', '؟', '-', ' ']
punc_matrix = []
for punc in punctuation:
    punc_matrix.append(extract_custom_char_embeddings(punc))
punc_matrix = np.array(punc_matrix)

def is_vector_punctuation(vec, threshold=1e-5):
    diff = np.linalg.norm(punc_matrix - vec, axis=1)
    return np.any(diff < threshold)

In [ ]:
class SparseCategoricalFocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, ignore_id=15, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma
        self.ignore_id = ignore_id

    def call(self, y_true, y_pred):
        y_true = tf.squeeze(y_true, axis=-1)
        y_true = tf.cast(y_true, tf.int32)

        mask = tf.cast(tf.not_equal(y_true, self.ignore_id), tf.float32)

        y_true_safe = tf.where(y_true == self.ignore_id, 0, y_true)

        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true_safe, y_pred, from_logits=False)

        p_t = tf.math.exp(-loss)
        focal_loss = tf.math.pow(1.0 - p_t, self.gamma) * loss

        masked_loss = focal_loss * mask

        return tf.reduce_sum(masked_loss) / (tf.reduce_sum(mask) + 1e-6)

In [ ]:
import h5py
import numpy as np
import tensorflow as tf

def multi_file_generator():
    for h5_path, lengths in files_config:
        with h5py.File(h5_path, 'r') as f:
            features_dset = f['features']
            labels_dset = f['tashkeel']

            current_idx = 0

            for seq_len in lengths:
                end_idx = current_idx + seq_len

                feat = features_dset[current_idx : end_idx]

                lbl = np.full((seq_len,), 15, dtype=np.int32)

                if feat.shape[0] > 0:
                    local_idx = 0

                    for feat_item in feat:
                        is_separator = np.all(feat_item[:768] == 0)

                        if is_separator:
                            lbl[local_idx] = 15
                            if local_idx > 0:
                                prev_emb = features_dset[current_idx + local_idx - 1]

                                if not is_vector_punctuation(prev_emb[768:1024]):
                                    lbl[local_idx - 1] = labels_dset[current_idx + local_idx - 1]

                        local_idx += 1

                    if not np.all(feat[-1][:768] == 0):
                        prev_emb = features_dset[end_idx - 1]

                        if not is_vector_punctuation(prev_emb[768:1024]):
                            lbl[-1] = labels_dset[end_idx - 1]

                    lbl = lbl.reshape(-1, 1)
                    yield feat, lbl

                current_idx = end_idx

dataset = tf.data.Dataset.from_generator(
    multi_file_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, INPUT_DIM), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.int32)
    )
)

In [ ]:
dataset = dataset.padded_batch(
    BATCH_SIZE,
    padding_values=(PADDING_INPUT, 15)
)
dataset = dataset.repeat()

dataset = dataset.prefetch(tf.data.AUTOTUNE)

def build_model():
    inputs = tf.keras.Input(shape=(None, INPUT_DIM))
    masked = tf.keras.layers.Masking(mask_value=PADDING_INPUT)(inputs)

    x = tf.keras.layers.Dense(HIDDEN_DIM * 2)(masked)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    residual = x

    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(HIDDEN_DIM, return_sequences=True))(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Add()([x, residual])
    residual = x

    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(HIDDEN_DIM, return_sequences=True))(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Add()([x, residual])
    residual = x

    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(HIDDEN_DIM, return_sequences=True))(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Add()([x, residual])

    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(HIDDEN_DIM, return_sequences=True))(x)

    x = tf.keras.layers.Dense(HIDDEN_DIM, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def masked_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true, axis=-1)
    y_true = tf.cast(y_true, tf.int32)

    mask = tf.cast(tf.not_equal(y_true, 15), tf.float32)

    y_pred_class = tf.cast(tf.argmax(y_pred, axis=-1), tf.int32)

    correct = tf.cast(tf.equal(y_true, y_pred_class), tf.float32)

    masked_correct = correct * mask

    return tf.reduce_sum(masked_correct) / (tf.reduce_sum(mask) + 1e-6)

In [ ]:
import h5py
import numpy as np

def another_smart_slicer_generator(lengths, offset=0):
    with h5py.File('/kaggle/input/arabic-diacritics-features-electra-val-4152/zizo_part_4152.h5', 'r') as f:
        features_dset = f['features']
        labels_dset = f['tashkeel']

        current_idx = offset

        for seq_len in lengths:
            end_idx = current_idx + seq_len

            feat = features_dset[current_idx : end_idx]
            raw_lbl = labels_dset[current_idx : end_idx]

            lbl = np.full((seq_len,), 15, dtype=np.int32)

            if feat.shape[0] > 0:
                local_idx = 0
                for feat_item in feat:
                    # Check if separator
                    if np.all(feat_item[:768] == 0):
                        lbl[local_idx] = 15
                        if local_idx > 0:
                            prev_emb = features_dset[current_idx + local_idx - 1]

                            if not is_vector_punctuation(prev_emb[768:1024]):
                                lbl[local_idx - 1] = labels_dset[current_idx + local_idx - 1]
                    local_idx += 1

                if not np.all(feat[-1][:768] == 0):
                    prev_emb = features_dset[end_idx - 1]
                    if not is_vector_punctuation(prev_emb[768:1024]):
                        lbl[-1] = labels_dset[end_idx - 1]

            lbl = lbl.reshape(-1, 1)
            yield feat, lbl

            current_idx = end_idx

In [ ]:
import pickle
sentences, tashkeel_sequences = load_data_pickle(padded_val)

new_sentences = sentences
new_tashkeel = tashkeel_sequences

# for i in range(len(sentences)):
#     text, tashkeel = remove_pads(sentences[i], tashkeel_sequences[i])
#     new_sentences.append(text)
#     new_tashkeel.append(tashkeel)

val_lengths = [len(s) for s in new_sentences]

In [ ]:
import math
val_dataset = tf.data.Dataset.from_generator(
    lambda: another_smart_slicer_generator(val_lengths, offset=0),
    output_signature=(
        tf.TensorSpec(shape=(None, INPUT_DIM), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.int32)
    )
)

val_dataset = val_dataset.padded_batch(
    BATCH_SIZE,
    padding_values=(PADDING_INPUT, 15)
)
val_dataset = val_dataset.repeat()


val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)
val_steps = math.ceil(len(val_lengths) / BATCH_SIZE)

In [ ]:
model = tf.keras.models.load_model(
    '/kaggle/working/last_char_electra_test.keras',
    custom_objects={
        'SparseCategoricalFocalLoss': SparseCategoricalFocalLoss,
        'masked_accuracy': masked_accuracy
    }
)

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=1,
    min_lr=1e-6,
    verbose=1
)

history = model.fit(
    dataset,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=15,
    initial_epoch=4,
    validation_data=val_dataset,
    validation_steps=val_steps,
    callbacks=[lr_callback],
    verbose=1
)

Epoch 5/15
2512/2512 ━━━━━━━━━━━━━━━━━━━━ 1611s 638ms/step - loss: 0.0352 - masked_accuracy: 0.9697 - val_loss: 0.0421 - val_masked_accuracy: 0.9685 - learning_rate: 0.0010
Epoch 6/15
2182/2512 ━━━━━━━━━━━━━━━━━━━━ 3:25 623ms/step - loss: 0.0323 - masked_accuracy: 0.9712

In [ ]:
# model = build_model()

# lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.5,
#     patience=1,
#     min_lr=1e-6,
#     verbose=1
# )

# model.compile(
#     optimizer='adam',
#     loss=SparseCategoricalFocalLoss(gamma=2.0),
#     metrics=[masked_accuracy]
# )

# model.fit(
#     dataset,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     epochs=15,
#     validation_data=val_dataset,
#     validation_steps=val_steps,
#     callbacks=[lr_callback],
#     verbose=1
# )

Epoch 1/15
2512/2512 ━━━━━━━━━━━━━━━━━━━━ 1615s 639ms/step - loss: 0.3353 - masked_accuracy: 0.8162 - val_loss: 0.0585 - val_masked_accuracy: 0.9540 - learning_rate: 0.0010
Epoch 2/15
2512/2512 ━━━━━━━━━━━━━━━━━━━━ 1604s 639ms/step - loss: 0.0606 - masked_accuracy: 0.9538 - val_loss: 0.0479 - val_masked_accuracy: 0.9628 - learning_rate: 0.0010
Epoch 3/15
2512/2512 ━━━━━━━━━━━━━━━━━━━━ 1604s 639ms/step - loss: 0.0466 - masked_accuracy: 0.9629 - val_loss: 0.0452 - val_masked_accuracy: 0.9655 - learning_rate: 0.0010
Epoch 4/15
2512/2512 ━━━━━━━━━━━━━━━━━━━━ 1605s 639ms/step - loss: 0.0396 - masked_accuracy: 0.9672 - val_loss: 0.0432 - val_masked_accuracy: 0.9676 - learning_rate: 0.0010
Epoch 5/15
   2/2512 ━━━━━━━━━━━━━━━━━━━━ 28:21 678ms/step - loss: 0.0395 - masked_accuracy: 0.9735

KeyboardInterrupt: 

# Testing

In [ ]:
import h5py

def another_smart_slicer_generator(lengths, offset=0):
    with h5py.File('/kaggle/input/arabic-diacrirization-features-really-val-4152/zizo_part_4152.h5', 'r') as f:
        features_dset = f['features']
        labels_dset = f['tashkeel']

        current_idx = offset

        for seq_len in lengths:
            end_idx = current_idx + seq_len

            feat = features_dset[current_idx : end_idx]
            lbl = labels_dset[current_idx : end_idx]
            is_space = np.all(feat[:, :768] == 0, axis=1)
            lbl[is_space] = 15

            lbl = lbl.reshape(-1, 1)

            yield feat, lbl

            current_idx = end_idx

In [ ]:
import pickle
sentences, tashkeel_sequences = load_data_pickle(padded_val)

new_sentences = sentences
new_tashkeel = tashkeel_sequences

# for i in range(len(sentences)):
#     text, tashkeel = remove_pads(sentences[i], tashkeel_sequences[i])
#     new_sentences.append(text)
#     new_tashkeel.append(tashkeel)

all_sentence_lengths = [len(s) for s in new_sentences]

In [ ]:
test_dataset = tf.data.Dataset.from_generator(
    lambda: another_smart_slicer_generator(all_sentence_lengths, offset=0),
    output_signature=(
        tf.TensorSpec(shape=(None, INPUT_DIM), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.int32)
    )
).padded_batch(BATCH_SIZE, padding_values=(PADDING_INPUT, 15))

In [ ]:
total_correct = 0
total_chars = 0
sentences_processed = 0

for batch_x, batch_y in test_dataset:

    batch_probs = model.predict_on_batch(batch_x)

    batch_pred_ids = np.argmax(batch_probs, axis=-1)

    batch_true_ids = batch_y.numpy().squeeze(axis=-1)

    batch_size = batch_pred_ids.shape[0]

    for k in range(batch_size):
        valid_indices = np.where(batch_true_ids[k] != -1)[0]

        if len(valid_indices) == 0:
            continue

        real_len = valid_indices[-1] + 1

        p_seq = batch_pred_ids[k][:real_len]
        t_seq = batch_true_ids[k][:real_len]
        mask = (t_seq != 15)
        matches = np.sum(p_seq[mask] == t_seq[mask])
        total_chars += np.sum(mask)


        total_correct += matches
        sentences_processed += 1
if total_chars > 0:
    print(f"Final Accuracy: {total_correct / total_chars:.2%}")

Final Accuracy: 97.68%


# Save the model

In [ ]:
model.save('last_char_electra_test.keras')

In [ ]:
import kagglehub
from kaggle_secrets import UserSecretsClient
import os

my_username = "ziad23samer"
my_key = "0dda25fbbd0112eac2ff1848742972a2"
os.environ["KAGGLE_USERNAME"] = my_username
os.environ["KAGGLE_KEY"] = my_key

handle = f"{my_username}/last_char_electra_test/keras/1"

local_model_dir = "/kaggle/working/"

kagglehub.model_upload(handle, local_model_dir, version_notes="Initial release from notebook")

Uploading Model https://www.kaggle.com/models/ziad23samer/last_char_electra_test/keras/1 ...
Model 'last_char_electra_test' does not exist or access is forbidden for user 'ziad23samer'. Creating or handling Model...
Model 'last_char_electra_test' Created.
Starting upload for file /kaggle/working/last_char_electra_test.keras


Uploading: 100%|██████████| 321M/321M [00:02<00:00, 119MB/s]  

Upload successful: /kaggle/working/last_char_electra_test.keras (306MB)
Starting upload for file /kaggle/working/.virtual_documents/__notebook_source__.ipynb



Uploading: 100%|██████████| 14.2k/14.2k [00:00<00:00, 62.6kB/s]

Upload successful: /kaggle/working/.virtual_documents/__notebook_source__.ipynb (14KB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/ziad23samer/last_char_electra_test/keras/1
